In [1]:
import numpy as np
import pandas as pd

In [2]:
image = pd.read_csv('../raw_data/train_image_level.csv')
study = pd.read_csv('../raw_data/train_study_level.csv')
df_image = image.copy()
df_study = study.copy()

In [3]:
print(df_image.shape)
df_image.head()

(6334, 4)


,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [4]:
print(df_study.shape)
df_study.head()

(6054, 5)


,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [5]:
df_study[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts()

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
0                       1                   0                         0                      2855
1                       0                   0                         0                      1676
0                       0                   1                         0                      1049
                                            0                         1                       474
dtype: int64

## Etude et merge des DF

In [6]:
df_image.isna().sum()

id                     0
boxes               2040
label                  0
StudyInstanceUID       0
dtype: int64

In [7]:
df_image['id'].duplicated().sum()

0

In [8]:
df_image['StudyInstanceUID'].duplicated().sum()

280

**/!\ Il y a 280 études** (StudyInstanceUID) **pour lesquels il y a 2 images dans l'étude**

In [9]:
df_study.isnull().sum()

id                          0
Negative for Pneumonia      0
Typical Appearance          0
Indeterminate Appearance    0
Atypical Appearance         0
dtype: int64

In [10]:
def clear_id_study(study_id): 
    study_id = study_id.replace('_study','')
    while study_id[0] == '0':
        study_id = study_id[1:]
    return study_id

In [11]:
df_study['StudyInstanceUID'] = df_study['id'].apply(clear_id_study)
df_study.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,StudyInstanceUID
0,00086460a852_study,0,1,0,0,86460a852
1,000c9c05fd14_study,0,0,0,1,c9c05fd14
2,00292f8c37bd_study,1,0,0,0,292f8c37bd
3,005057b3f880_study,1,0,0,0,5057b3f880
4,0051d9b12e72_study,0,0,0,1,51d9b12e72


In [12]:
df = df_image.merge(df_study.drop(columns='id'), on ='StudyInstanceUID', how='left')
print(df.shape)
df.head()

(6334, 8)


,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0.0,1.0,0.0,0.0
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,1.0,0.0,0.0,0.0
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0.0,1.0,0.0,0.0
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0.0,0.0,0.0,1.0
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0.0,1.0,0.0,0.0


In [13]:
df.isna().sum()

id                             0
boxes                       2040
label                          0
StudyInstanceUID               0
Negative for Pneumonia       386
Typical Appearance           386
Indeterminate Appearance     386
Atypical Appearance          386
dtype: int64

In [14]:
df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts().sum()

5948

In [15]:
df[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts()

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
0.0                     1.0                 0.0                       0.0                    2842
1.0                     0.0                 0.0                       0.0                    1617
0.0                     0.0                 1.0                       0.0                    1039
                                            0.0                       1.0                     450
dtype: int64

## Traitement des NaN

In [16]:
df['boxes'].isna().sum()

2040

In [17]:
df_nan = df.loc[df['boxes'].isna() == True ]
print(df_nan[['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']].value_counts())
print(df_nan['label'].value_counts())
df_nan.head(5)

Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  Atypical Appearance
1.0                     0.0                 0.0                       0.0                    1617
0.0                     1.0                 0.0                       0.0                     146
                        0.0                 0.0                       1.0                      85
                                            1.0                       0.0                      47
dtype: int64
none 1 0 0 1 1    2040
Name: label, dtype: int64


,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,1.0,0.0,0.0,0.0
6,0023f02ae886_image,NaN,none 1 0 0 1 1,2fa400b873f5,1.0,0.0,0.0,0.0
22,00c1515729a8_image,NaN,none 1 0 0 1 1,1a58b43cf286,0.0,1.0,0.0,0.0
24,00e3a7e91a34_image,NaN,none 1 0 0 1 1,74ba8f2badcb,1.0,0.0,0.0,0.0
26,010214f9b600_image,NaN,none 1 0 0 1 1,2a7b4e669dda,1.0,0.0,0.0,0.0


**/!\ Les 1617 'Negative for Pneumonia' ont tous NaN dans la colonne 'boxes', mais pas tous les NaN correspondent à 'Negative for Pneumonia'**

Aussi, toutes les boxes NaN ont un label associé 'none 1 0 0 1 1'